<a href="https://colab.research.google.com/github/robertosgpontes/osem/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Formulação do Problema

O problema apresentado pode ser formulado através de uma

\begin{equation}
	\max  \sum_{i=1}^{n}{[A_{i}^1-(A_{i}^2-A_{i}^3)]X_i}
\end{equation}
e
\begin{equation}
\min \sum_{i=1}^{n}{C_iX_i}
\end{equation}


Sujeito a
\begin{equation}
x_1 \leq b_1
\end{equation}
\begin{equation}
x_2 \leq b_2
\end{equation}
\begin{equation}
x_3 \leq b_3
\end{equation}
\begin{equation}
x_1 + x_3 \leq b_4
\end{equation}
\begin{equation}
x_1 + x_2 + x_3 + x_4 \leq b_5
\end{equation}
\begin{equation}
x_1 \geq b_6
\end{equation}
\begin{equation}
x_2 \geq b_7
\end{equation}
\begin{equation}
x_1, x_2, x_3, x_4 \geq 0
\end{equation}


In [ ]:
% pip install pulp

In [ ]:
# Python libraries used to solve the problem Linear Programing
import pulp
import numpy as np
import pandas as pd

In [ ]:
# Variáveis do modelo

A1 = np.array([11.444, 0.047, 1.523, 0.095])
A2 = np.array([0.899, 0.0, 0.926, 0.073]) 
A3 = np.array([0.00106, 0.00010, 0.00021, 0.00006])
C = np.array([1.41, 4.99, 2.41, 6.24])
A = np.array([  [1, 0, 0, 0],
                [0, 0, 1, 0],
                [0, 0, 0, 1],
                [1, 0, 1, 0],
                [1, 1, 1, 1],
                [1, 0, 0, 0],
                [0, 1, 0, 0]])

b = np.array([[557],[1143],[1577],[1700],[10310],[160],[6990]])

In [ ]:
def create_x_variables():
    x1 = pulp.LpVariable("x1",lowBound = 0) 
    x2 = pulp.LpVariable("x2",lowBound = 0)
    x3 = pulp.LpVariable("x3",lowBound = 0) 
    x4 = pulp.LpVariable("x4",lowBound = 0) 
    return np.array([[x1], [x2], [x3], [x4]]) 

In [ ]:
def load_model(lpm, A, X, b):
    Ax = A.dot(X)

    for i in range(0,4):
        lpm += Ax[i,0] <= b[i,0]

    lpm += Ax[4,0] == b[4,0]

    for i in range(5,7):
        lpm += Ax[i,0] >= b[i,0]

    return lpm

# Approach 1: Solution with Hierarchical Methodology

Maximizing for one objective, then adding it as a constraint and solving for the other objective

## 1.1. First Step

\begin{equation}
S_1 = \max  \sum_{i=1}^{n}{[A_{i}^1-(A_{i}^2-A_{i}^3)]X_i}
\end{equation}

Sujeito a
\begin{equation}
x_1 \leq b_1
\end{equation}
\begin{equation}
x_2 \leq b_2
\end{equation}
\begin{equation}
x_3 \leq b_3
\end{equation}
\begin{equation}
x_1 + x_3 \leq b_4
\end{equation}
\begin{equation}
x_1 + x_2 + x_3 + x_4 \leq b_5
\end{equation}
\begin{equation}
x_1 \geq b_6
\end{equation}
\begin{equation}
x_2 \geq b_7
\end{equation}
\begin{equation}
x_1, x_2, x_3, x_4 \geq 0
\end{equation}


In [ ]:
linearProblem = pulp.LpProblem("First_Objective_Max",pulp.LpMaximize)

X = create_x_variables()

f1 = (A1 - (A2 + A3))*X.T

linearProblem += f1.sum()

linearProblem = load_model(linearProblem, A, X, b)

linearProblem.writeLP('modelStep1.lp')

solution = linearProblem.solve()

f1_max = pulp.value(linearProblem.objective)

In [ ]:
# print(linearProblem)

First_Objective_Max:
MAXIMIZE
10.543940000000001*x1 + 0.0469*x2 + 0.5967899999999998*x3 + 0.02194*x4 + 0.0
SUBJECT TO
_C1: x1 <= 557

_C2: x3 <= 1143

_C3: x4 <= 1577

_C4: x1 + x3 <= 1700

_C5: x1 + x2 + x3 + x4 = 10310

_C6: x1 >= 160

_C7: x2 >= 6990

VARIABLES
x1 Continuous
x2 Continuous
x3 Continuous
x4 Continuous



In [ ]:
print("#1 - " + str(pulp.LpStatus[solution])+" ; max value = "+str(pulp.value(linearProblem.objective))+
      " ; x1_opt = "+str(pulp.value(X[0][0]))+
      " ; x2_opt = "+str(pulp.value(X[1][0]))+
      " ; x3_opt = "+str(pulp.value(X[2][0]))+
      " ; x4_opt = "+str(pulp.value(X[3][0])))

#1 - Optimal ; max value = 6958.914550000001 ; x1_opt = 557.0 ; x2_opt = 8610.0 ; x3_opt = 1143.0 ; x4_opt = 0.0


## 1.1. Second Step


\begin{equation}
\min \sum_{i=1}^{n}{C_iX_i}
\end{equation}
Sujeito a
\begin{equation}
x_1 \leq b_1
\end{equation}
\begin{equation}
x_2 \leq b_2
\end{equation}
\begin{equation}
x_3 \leq b_3
\end{equation}
\begin{equation}
x_1 + x_3 \leq b_4
\end{equation}
\begin{equation}
x_1 + x_2 + x_3 + x_4 \leq b_5
\end{equation}
\begin{equation}
x_1 \geq b_6
\end{equation}
\begin{equation}
x_2 \geq b_7
\end{equation}
\begin{equation}
\sum_{i=1}^{n}{[A_{i}^1-(A_{i}^2-A_{i}^3)]X_i} \geq S_1
\end{equation}
\begin{equation}
x_1, x_2, x_3, x_4 \geq 0
\end{equation}


In [ ]:
linearProblem2 = pulp.LpProblem("Second_Objective_Min",pulp.LpMaximize)

X = create_x_variables()

linearProblem2 += C.dot(X)[0]

linearProblem2 = load_model(linearProblem2, A, X, b)

f1 = (A1 - (A2 + A3))*X.T

linearProblem2 += f1.sum() >= f1_max

linearProblem2.writeLP('modelStep2.lp')

solution2 = linearProblem2.solve()

In [ ]:
print(linearProblem2)

Second_Objective_Min:
MAXIMIZE
1.41*x1 + 4.99*x2 + 2.41*x3 + 6.24*x4 + 0.0
SUBJECT TO
_C1: x1 <= 557

_C2: x3 <= 1143

_C3: x4 <= 1577

_C4: x1 + x3 <= 1700

_C5: x1 + x2 + x3 + x4 = 10310

_C6: x1 >= 160

_C7: x2 >= 6990

_C8: 10.54394 x1 + 0.0469 x2 + 0.59679 x3 + 0.02194 x4 >= 6958.91455

VARIABLES
x1 Continuous
x2 Continuous
x3 Continuous
x4 Continuous



In [ ]:
print(str(pulp.LpStatus[solution2])+" ; max value = "+str(pulp.value(linearProblem2.objective))+
      " ; x1_opt = "+str(pulp.value(X[0][0]))+
      " ; x2_opt = "+str(pulp.value(X[1][0]))+
      " ; x3_opt = "+str(pulp.value(X[2][0]))+
      " ; x4_opt = "+str(pulp.value(X[3][0])))

Optimal ; max value = 46503.90000000035 ; x1_opt = 557.0 ; x2_opt = 8610.0 ; x3_opt = 1143.0 ; x4_opt = 5.6388672e-11


# 2. Solution with ...

\begin{equation}
	\max  \alpha\sum_{i=1}^{n}{[A_{i}^1-(A_{i}^2-A_{i}^3)]X_i} - (1-\alpha)\sum_{i=1}^{n}{C_iX_i}
\end{equation}

Sujeito a
\begin{equation}
x_1 \leq b_1
\end{equation}
\begin{equation}
x_2 \leq b_2
\end{equation}
\begin{equation}
x_3 \leq b_3
\end{equation}
\begin{equation}
x_1 + x_3 \leq b_4
\end{equation}
\begin{equation}
x_1 + x_2 + x_3 + x_4 \leq b_5
\end{equation}
\begin{equation}
x_1 \geq b_6
\end{equation}
\begin{equation}
x_2 \geq b_7
\end{equation}
\begin{equation}
x_1, x_2, x_3, x_4 \geq 0
\end{equation}

In [ ]:
def f_alpha(alpha, f1, f2):
  return (alpha*f1 + (1-alpha)*f2)

In [ ]:
def run(f_alphas):
  solution = []
  i = 0
  for alpha in f_alphas:
    model = pulp.LpProblem("MultiObjetivo",pulp.LpMaximize)

    X = create_x_variables()

    f1 = (A1 - (A2 + A3))*X.T
    f1 = f1.sum()

    f2 = C.dot(X)[0]

    model += f_alpha(1, f1, f2)

    model = load_model(model, A, X, b)

    solution = model.solve()

    solution.append([i,
                    alpha,
                    str(pulp.LpStatus[solution]),
                    pulp.value(X[0][0]), 
                    pulp.value(X[1][0]), 
                    pulp.value(X[2][0]), 
                    pulp.value(X[3][0]),
                    pulp.value(model.objective)])
    i += 1
    
  return pd.DataFrame(solution, columns=["iter","alpha","status","x1","x2", "x3", "x4", "obj_value"])

In [ ]:
f_alfa

In [ ]:
solutionTable = 

In [ ]:
print(str(pulp.LpStatus[solution2])+" ; max value = "+str(pulp.value(linearProblem2.objective))+
      " ; x1_opt = "+str(pulp.value(X[0][0]))+
      " ; x2_opt = "+str(pulp.value(X[1][0]))+
      " ; x3_opt = "+str(pulp.value(X[2][0]))+
      " ; x4_opt = "+str(pulp.value(X[3][0])))